In [ ]:
#| default_exp blup

In [ ]:
#| export

import jax
import jax.numpy as jnp
import numpy as np
from functools import partial
from typing import Tuple
from scipy.sparse import coo_matrix

# JAX's iterative solver and sparse matrix format
from jax.scipy.sparse.linalg import cg
import jax.experimental.sparse as jsparse

# Assume the 'chewc' library is installed
from chewc.structs import (
    Population,
    Trait,
    GeneticMap,
    quick_haplo,
    add_trait
)
from chewc.pheno import calculate_phenotypes
from chewc.select import select_top_k
from chewc.cross import random_mating, cross_pair

# --- Scalable Core Functions ---

def build_a_inverse_sparse(pedigree_meta: np.ndarray) -> jsparse.BCOO:
    """
    Constructs the inverse of the Numerator Relationship Matrix (A_inv)
    directly from a pedigree using the correct, numerically stable version of
    Henderson's rules. Returns the matrix in JAX's BCOO sparse format.
    """
    n = pedigree_meta.shape[0]
    # Pre-allocate with a safe upper bound on non-zero elements
    max_elements = n * 9  # Max of 9 contributions per animal with two parents
    data, rows, cols = (
        np.zeros(max_elements, dtype=np.float32),
        np.zeros(max_elements, dtype=np.int32),
        np.zeros(max_elements, dtype=np.int32),
    )
    ptr = 0

    def add_element(r, c, val):
        nonlocal ptr
        if ptr >= max_elements:
            raise ValueError("Exceeded pre-allocated memory for sparse matrix.")
        rows[ptr], cols[ptr], data[ptr] = r, c, val
        ptr += 1

    # Loop through individuals and add their contributions to A_inv
    for i in range(n):
        sire = int(pedigree_meta[i, 1])
        dam = int(pedigree_meta[i, 2])

        if sire == -1 and dam == -1:  # Founder
            add_element(i, i, 1.0)
        elif sire == -1 or dam == -1:  # One parent known
            parent = sire if sire != -1 else dam
            add_element(i, i, 4/3)
            add_element(i, parent, -2/3)
            add_element(parent, i, -2/3)
            add_element(parent, parent, 1/3)
        else:  # Both parents known
            add_element(i, i, 2.0)
            add_element(i, sire, -1.0)
            add_element(sire, i, -1.0)
            add_element(i, dam, -1.0)
            add_element(dam, i, -1.0)
            add_element(sire, sire, 0.5)
            add_element(dam, dam, 0.5)
            add_element(sire, dam, 0.5)
            add_element(dam, sire, 0.5)

    # Use scipy to efficiently sum duplicate entries
    scipy_coo = coo_matrix((data[:ptr], (rows[:ptr], cols[:ptr])), shape=(n, n))
    scipy_coo.sum_duplicates()

    # **DEBUG**: Check for invalid values before returning
    if not np.all(np.isfinite(scipy_coo.data)):
        print("[DEBUG] WARNING: Non-finite values (NaN or inf) found in sparse A_inv data!")
    else:
        print(f"[DEBUG] Sparse A_inv successfully created with {len(scipy_coo.data)} non-zero elements.")
        print(f"[DEBUG] First 5 data points of A_inv: {scipy_coo.data[:5]}")

    # The format requires indices of shape (nse, 2)
    indices = np.stack([scipy_coo.row, scipy_coo.col], axis=1)

    return jsparse.BCOO((jnp.array(scipy_coo.data), jnp.array(indices)), shape=(n, n))


@jax.jit
def build_g_matrix(geno: jnp.ndarray) -> jnp.ndarray:
    """Constructs the Genomic Relationship Matrix (G)."""
    n_ind, _, _, _ = geno.shape
    dosages = jnp.sum(geno, axis=2, dtype=jnp.float32).reshape(n_ind, -1)
    p = jnp.mean(dosages, axis=0) / 2.0
    p = jnp.clip(p, 1e-6, 1.0 - 1e-6)
    Z = dosages - 2 * p
    k = 2 * jnp.sum(p * (1 - p))
    return (Z @ Z.T) / k

@partial(jax.jit, static_argnames=('n_traits',))
def solve_multi_trait_mme_iterative(
    y: jnp.ndarray,
    relationship_matrix_inv: jsparse.BCOO,
    G0_inv: jnp.ndarray,
    R0_inv: jnp.ndarray,
    n_traits: int
) -> jnp.ndarray:
    """Solves a multi-trait MME using an iterative Conjugate Gradient solver."""
    n_ind = relationship_matrix_inv.shape[0]

    def lhs_matvec(v_stacked):
        b = v_stacked[:n_traits]
        u = v_stacked[n_traits:].reshape((n_ind, n_traits), order='F')
        
        # Top block
        top = (R0_inv @ b) * n_ind + R0_inv @ u.sum(axis=0)
        
        # Bottom block
        Zt_R_inv_X_b = (jnp.ones((n_ind, 1)) @ b.reshape(1, -1)) @ R0_inv.T
        Zt_R_inv_Z_u = u @ R0_inv.T
        G_inv_u = (relationship_matrix_inv @ u) @ G0_inv.T
        bottom = (Zt_R_inv_X_b + Zt_R_inv_Z_u + G_inv_u).flatten('F')
        
        return jnp.concatenate([top, bottom])

    RHS = jnp.concatenate([R0_inv @ y.sum(axis=0), (y @ R0_inv.T).flatten('F')])
    
    solutions, _ = cg(lhs_matvec, RHS, maxiter=1000)
    
    return solutions[n_traits:].reshape((n_ind, n_traits), order='F')


def remap_pedigree(full_pedigree: jnp.ndarray) -> np.ndarray:
    """Remaps IDs and returns a NumPy array for the sparse builder."""
    original_ids = full_pedigree[:, 0]
    id_map = {int(og_id): i for i, og_id in enumerate(original_ids)}
    id_map[-1] = -1
    
    remapped_pedigree = np.zeros(full_pedigree.shape, dtype=np.int32)
    remapped_pedigree[:, 0] = np.arange(len(original_ids))
    remapped_pedigree[:, 1] = [id_map[int(i)] for i in full_pedigree[:, 1]]
    remapped_pedigree[:, 2] = [id_map[int(i)] for i in full_pedigree[:, 2]]
    return remapped_pedigree


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()